In [18]:
import pandas as pd
import numpy as np
import scipy.stats as ss

In [19]:
# Исходная таблица
categorical_1 = ['Тормозят', 'Притормаживают', 'Не тормозят']
categorical_2 = ['М', 'Ж']
observed_df=pd.DataFrame(
    data= 
    [
        [20, 15], 
        [11, 12], 
        [7, 9]
    ],
    index=categorical_1,
    columns=categorical_2
)

observed_df

,М,Ж
Тормозят,20,15
Притормаживают,11,12
Не тормозят,7,9


In [20]:
# Расширенная таблица с суммами по строкам и столбцам
observed_sum_df = observed_df.copy(deep=True)
observed_sum_df['row_sum'] = observed_sum_df.sum(axis=1)
observed_sum_df.loc['col_sum'] = observed_sum_df.sum(axis=0)

observed_sum_df

,М,Ж,row_sum
Тормозят,20,15,35
Притормаживают,11,12,23
Не тормозят,7,9,16
col_sum,38,36,74


In [21]:
# Частоты по строкам и столбцам
row_sum = observed_sum_df.row_sum[0:-1]
col_sum = observed_sum_df.loc['col_sum'][0:-1]
# Размер исходной таблицы
r, c = observed_df.shape
# Сумма частот
N = observed_sum_df.row_sum[-1]
# Таблица с модельными значениями
expected_df = observed_df.copy(deep=True)
expected_df.iloc[:] = 1
expected_df = expected_df.mul(col_sum).mul(row_sum, axis=0).mul(1/N)

expected_df

,М,Ж
Тормозят,17.972973,17.027027
Притормаживают,11.810811,11.189189
Не тормозят,8.216216,7.783784


In [22]:
# Расстояние Пирсона
X2 = ((observed_df - expected_df).pow(2) 
      / expected_df
     ).to_numpy().sum()

X2

0.9544070774762995

In [23]:
# Степени свободы
df = (r - 1)*(c - 1)
df

2

In [24]:
# p-value
p_value = 1 - ss.chi2.cdf(x=X2, df=df)

p_value

0.6205162173513056

In [25]:
# Свёрнуто в функцию
def chi2_test(observed_df):
    '''
    Функция считает p-value для chi-squared independence test

    Parameters
    ----------
    observed_df: pandas.DataFrame
        Таблица с наблюдаемыми исходами

    Returns
    -------
    p_value: float
        Значение p-value для теста Хи-квадрат
    '''
    # Расширенная таблица с суммами по строкам и столбцам
    observed_sum_df = observed_df.copy(deep=True)
    observed_sum_df['row_sum'] = observed_sum_df.sum(axis=1)
    observed_sum_df.loc['col_sum'] = observed_sum_df.sum(axis=0)

    # Частоты по строкам и столбцам
    row_sum = observed_sum_df.row_sum[0:-1]
    col_sum = observed_sum_df.loc['col_sum'][0:-1]
    # Размер исходной таблицы
    r, c = observed_df.shape
    # Сумма частот
    N = observed_sum_df.row_sum[-1]
    # Таблица с модельными значениями
    expected_df = observed_df.copy(deep=True)
    expected_df.iloc[:] = 1
    expected_df = expected_df.mul(col_sum).mul(row_sum, axis=0).mul(1/N)

    # Расстояние Пирсона
    X2 = ((observed_df - expected_df).pow(2) 
          / expected_df
         ).to_numpy().sum()
    
    # Степени свободы
    df = (r - 1)*(c - 1)
    
    # p-value
    p_value = 1 - ss.chi2.cdf(x=X2, df=df)
    
    return p_value

In [26]:
# Исходная таблица
categorical_1 = ['Тормозят', 'Притормаживают', 'Не тормозят']
categorical_2 = ['М', 'Ж']
observed_df=pd.DataFrame(
    data= 
    [
        [20, 15], 
        [11, 12], 
        [7, 9]
    ],
    index=categorical_1,
    columns=categorical_2
)

observed_df

,М,Ж
Тормозят,20,15
Притормаживают,11,12
Не тормозят,7,9


In [27]:
# p-value
chi2_test(observed_df)

0.6205162173513056

In [28]:
# Проверка
ss.chi2_contingency(observed_df)[1]

0.6205162173513055

In [29]:
# Пример №2
# Таблица сопржённости с поведением пользователей после рекламной кампании
contingency_table = pd.DataFrame({
    'Men': [21, 75, 5],
    'Wmen': [16, 60, 10]})
contingency_table
# 0 - Увеличивается сумма покупки
# 1 - Не меняется сумма покупки
# 3 - Уменьшается сумма покупки

# Является ли рекламная компания эффективной?
ss.chi2_contingency(contingency_table)
# => не является

(2.8239706434502656,
 0.24365906101675808,
 2,
 array([[19.98395722, 17.01604278],
        [72.9144385 , 62.0855615 ],
        [ 8.10160428,  6.89839572]]))